![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/)

# RENerChunksFilter

The `RENerChunksFilter` annotator filters NER chunks that contains the desired NER entity pairs only.

**📖 Learning Objectives:**

1. Understand how to filters desired relation pairs for the `RelationExtractionDLModel`

2. Become comfortable using the different parameters of the annotator.

**🔗 Helpful Links:**

* For extended examples of usage, see the [Spark NLP Workshop](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/healthcare-nlp/03.0.Clinical_Relation_Extraction.ipynb)

* Python Documentation: [RENerChunksFilter](https://nlp.johnsnowlabs.com/licensed/api/python/reference/autosummary/sparknlp_jsl/annotator/re/relation_ner_chunk_filter/index.html#sparknlp_jsl.annotator.re.relation_ner_chunk_filter.RENerChunksFilter.name)

* Scala Documentation: [RENerChunksFilter](https://nlp.johnsnowlabs.com/licensed/api/com/johnsnowlabs/nlp/annotators/re/RENerChunksFilter.html)

* Relation Extraction Models and Relation Pairs Table: [In this link](https://nlp.johnsnowlabs.com/docs/en/best_practices_pretrained_models#relation-extraction-models-and-relation-pairs-table). Contains the available Relation Extraction models, its labels, associated NER model, and meaningful relation pairs to filter.

## **📜 Background**


Filtering the possible relations can be useful to perform additional analysis for a specific use case (e.g., checking adverse drug reactions and drug realations), which can be the input for further analysis using a pretrained `RelationExtractionDLModel`.

This annotator filters the NER chunks that contains the desired relation pairs, excluding the annotations between other entities.

## **🎬 Colab Setup**

In [1]:
# Install the johnsnowlabs library to access Spark-NLP for Healthcare
! pip install -q johnsnowlabs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.2/126.2 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.3/547.3 kB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 665.7/665.7 kB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.4/95.4 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 62.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.7 M

In [2]:
# Upload license keys
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

Please Upload your John Snow Labs License using the button below


Saving spark_nlp_for_healthcare_spark_ocr_8734.json to spark_nlp_for_healthcare_spark_ocr_8734.json


In [3]:
# Install licensed libraries
from johnsnowlabs import nlp


nlp.install()

👌 Detected license file /content/spark_nlp_for_healthcare_spark_ocr_8734.json
🚨 Outdated Medical Secrets in license file. Version=5.2.0 but should be Version=5.2.1
🚨 Outdated OCR Secrets in license file. Version=5.1.0 but should be Version=5.1.2
👷 Trying to install compatible secrets. Use nlp.settings.enforce_versions=False if you want to install outdated secrets.
📋 Stored John Snow Labs License in /root/.johnsnowlabs/licenses/license_number_0_for_Spark-Healthcare_Spark-OCR.json
👷 Setting up  John Snow Labs home in /root/.johnsnowlabs, this might take a few minutes.
🙆 JSL Home setup in /root/.johnsnowlabs
👌 Detected license file /content/spark_nlp_for_healthcare_spark_ocr_8734.json
👷 Trying to install compatible secrets. Use nlp.settings.enforce_versions=False if you want to install outdated secrets.
Installing /root/.johnsnowlabs/py_installs/spark_nlp_jsl-5.2.1-py3-none-any.whl to /usr/bin/python3
Installed 1 products:
💊 Spark-Healthcare==5.2.1 installed! ✅ Heal the planet with NLP! 


In [4]:
from johnsnowlabs import nlp, medical
import pandas as pd
from pyspark.sql import DataFrame
import pyspark.sql.functions as F
import pyspark.sql.types as T


# Start Spark Session
spark = nlp.start()
spark

👌 Detected license file /content/spark_nlp_for_healthcare_spark_ocr_8734.json
👷 Trying to install compatible secrets. Use nlp.settings.enforce_versions=False if you want to install outdated secrets.
👌 Launched cpu optimized session with with: 🚀Spark-NLP==5.2.2, 💊Spark-Healthcare==5.2.1, running on ⚡ PySpark==3.4.0


## **🖨️ Input/Output Annotation Types**
- Input: `CHUNK, DEPENDENCY`
- Output: `CHUNK`

## **🔎 Parameters**


- `maxSyntacticDistance` *(Int)*: Maximum syntactic distance between a pair of named entities to consider them as a relation. Increasing this value will increase recall, but also increase the number of false positives.

- `relationPairs` *(List[Str])*: List of dash-separated pairs of named entities. For example, [“Biomarker-RelativeDay”] will process all relations between entities of type “Biomarker” and “RelativeDay”.

- `relationPairsCaseSensitive` *(Boolean)*: Determines whether relation pairs are case sensitive.


### `setMaxSyntacticDistance`

This parameter is used for setting the maximal syntactic distance, as threshold.

The precision of the RE model is controlled by "maxSyntacticDistance", which sets the maximum syntactic distance between named entities, controling the pairs to be classified. A larger value will improve recall at the expense at lower precision.

Let's compare the result with different values on an sample text.

First, we set the `.maxSyntacticDistance` to `10`.

In [6]:
documenter = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentencer = nlp.SentenceDetector()\
    .setInputCols(["document"])\
    .setOutputCol("sentences")

tokenizer = nlp.Tokenizer()\
    .setInputCols(["sentences"])\
    .setOutputCol("tokens")

words_embedder = nlp.WordEmbeddingsModel()\
    .pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("embeddings")

pos_tagger = nlp.PerceptronModel()\
    .pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("pos_tags")

ner_tagger = medical.NerModel.pretrained("ner_ade_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")

ner_chunker = medical.NerConverter()\
    .setInputCols(["sentences", "tokens", "ner_tags"])\
    .setOutputCol("ner_chunks")

dependency_parser = nlp.DependencyParserModel()\
    .pretrained("dependency_conllu", "en")\
    .setInputCols(["sentences", "pos_tags", "tokens"])\
    .setOutputCol("dependencies")

ade_re_ner_chunk_filter = medical.RENerChunksFilter() \
    .setInputCols(["ner_chunks", "dependencies"])\
    .setOutputCol("re_ner_chunks")\
    .setMaxSyntacticDistance(10)

ade_re_model = medical.RelationExtractionDLModel()\
    .pretrained('redl_ade_biobert', 'en', "clinical/models") \
    .setInputCols(["re_ner_chunks", "sentences"]) \
    .setPredictionThreshold(0.5)\
    .setOutputCol("relations")

pipeline = nlp.Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer,
    words_embedder,
    pos_tagger,
    ner_tagger,
    ner_chunker,
    dependency_parser,
    ade_re_ner_chunk_filter,
    ade_re_model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[OK!]
ner_ade_clinical download started this may take some time.
[OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[OK!]
redl_ade_biobert download started this may take some time.
[OK!]


In [8]:
example ="""A 44-year-old man taking naproxen for chronic low back pain and a 20-year-old woman on oxaprozin for rheumatoid arthritis presented with tense bullae and cutaneous fragility on the face and the back of the hands."""

spark_df = spark.createDataFrame([[example]]).toDF("text")

results = model.transform(spark_df)

In [11]:
results.select(
    F.explode(F.arrays_zip(results.relations.result, results.relations.metadata)).alias("cols")
).select(
    F.expr("cols['0']").alias("relation"),
    F.expr("cols['1'].chunk1").alias("chunk1"),
    F.expr("cols['1'].entity1").alias("label1"),
    F.expr("cols['1'].chunk2").alias("chunk2"),
    F.expr("cols['1'].entity2").alias("label2"),
    F.expr("cols['1'].syntactic_distance ").alias("syntactic_distance ")
).show()

+--------+------------+------+--------------------+------+-------------------+
|relation|      chunk1|label1|              chunk2|label2|syntactic_distance |
+--------+------------+------+--------------------+------+-------------------+
|       1|    naproxen|  DRUG|           oxaprozin|  DRUG|                  3|
|       1|    naproxen|  DRUG|        tense bullae|   ADE|                  6|
|       1|    naproxen|  DRUG|cutaneous fragili...|   ADE|                  7|
|       1|   oxaprozin|  DRUG|        tense bullae|   ADE|                  3|
|       1|   oxaprozin|  DRUG|cutaneous fragili...|   ADE|                  4|
|       1|tense bullae|   ADE|cutaneous fragili...|   ADE|                  3|
+--------+------------+------+--------------------+------+-------------------+



Now, we will set the `maxSyntacticDistance` to `4` and see the difference from the previous results.

In [12]:
ade_re_ner_chunk_filter = medical.RENerChunksFilter() \
    .setInputCols(["ner_chunks", "dependencies"])\
    .setOutputCol("re_ner_chunks")\
    .setMaxSyntacticDistance(4)

pipeline = nlp.Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer,
    words_embedder,
    pos_tagger,
    ner_tagger,
    ner_chunker,
    dependency_parser,
    ade_re_ner_chunk_filter,
    ade_re_model
])

model = pipeline.fit(empty_data)

In [13]:
results = model.transform(spark_df)

In [14]:
results.select(
    F.explode(F.arrays_zip(results.relations.result, results.relations.metadata)).alias("cols")
).select(
    F.expr("cols['0']").alias("relation"),
    F.expr("cols['1'].chunk1").alias("chunk1"),
    F.expr("cols['1'].entity1").alias("label1"),
    F.expr("cols['1'].chunk2").alias("chunk2"),
    F.expr("cols['1'].entity2").alias("label2"),
    F.expr("cols['1'].syntactic_distance ").alias("syntactic_distance ")
).show()

+--------+------------+------+--------------------+------+-------------------+
|relation|      chunk1|label1|              chunk2|label2|syntactic_distance |
+--------+------------+------+--------------------+------+-------------------+
|       1|    naproxen|  DRUG|           oxaprozin|  DRUG|                  3|
|       1|   oxaprozin|  DRUG|        tense bullae|   ADE|                  3|
|       1|   oxaprozin|  DRUG|cutaneous fragili...|   ADE|                  4|
|       1|tense bullae|   ADE|cutaneous fragili...|   ADE|                  3|
+--------+------------+------+--------------------+------+-------------------+



We can see that the relations with syntactic distance greater than 4 were removed from the results. In this case, as we had two entities close to each other, having a higher maximum syntactic distance was creating false positives, and when controlling this parameter we were able to obtain the expected result.

### `setRelationPairs`


This parameter is used for setting the list of dash-separated pairs of named entities. For example, `["drug-ade"]` will process all relations between entities of type "drug" and "ade".

Now, we will set `setRelationPairs(["OCCURRENCE-DURATION"])` parameter and expect only "OCCURRENCE-DURATION" and "DURATION-OCCURENCE" relations.

In [15]:
ner_tagger_clinical = medical.NerModel.pretrained("ner_events_clinical", "en", "clinical/models")\
    .setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("ner_tags")

clinical_re_ner_chunk_filter = medical.RENerChunksFilter() \
    .setInputCols(["ner_chunks", "dependencies"])\
    .setOutputCol("re_ner_chunks")\
    .setMaxSyntacticDistance(10)\
    .setRelationPairs(["OCCURRENCE-DURATION"])

clinical_re_model = medical.RelationExtractionDLModel()\
    .pretrained('redl_temporal_events_biobert', 'en', "clinical/models") \
    .setInputCols(["re_ner_chunks", "sentences"]) \
    .setPredictionThreshold(0.5)\
    .setOutputCol("relations")


pipeline = nlp.Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer,
    words_embedder,
    pos_tagger,
    ner_tagger_clinical,
    ner_chunker,
    dependency_parser,
    clinical_re_ner_chunk_filter,
    clinical_re_model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

ner_events_clinical download started this may take some time.
[OK!]
redl_temporal_events_biobert download started this may take some time.
[OK!]


In [17]:
example = """The patient is a 68-year-old Caucasian male with past medical history of diabetes mellitus.
He was doing fairly well until last week while mowing the lawn, he injured his right foot. """

spark_df = spark.createDataFrame([[example]]).toDF("text")
results = model.transform(spark_df)

Lets show the OCCURRENCE-DURATION relations by using pandas dataframe.

In [18]:
results.select(
    F.explode(F.arrays_zip(results.relations.result, results.relations.metadata)).alias("cols")
).select(
    F.expr("cols['0']").alias("relation"),
    F.expr("cols['1'].chunk1").alias("chunk1"),
    F.expr("cols['1'].entity1").alias("label1"),
    F.expr("cols['1'].chunk2").alias("chunk2"),
    F.expr("cols['1'].entity2").alias("label2"),
    F.expr("cols['1'].syntactic_distance ").alias("syntactic_distance ")
).show()

+--------+-----------------+----------+---------------+----------+-------------------+
|relation|           chunk1|    label1|         chunk2|    label2|syntactic_distance |
+--------+-----------------+----------+---------------+----------+-------------------+
|  BEFORE|doing fairly well|OCCURRENCE|      last week|  DURATION|                  3|
| OVERLAP|        last week|  DURATION|mowing the lawn|OCCURRENCE|                  3|
+--------+-----------------+----------+---------------+----------+-------------------+



As seen above, we only have "OCCURRENCE-DURATION" relations.

### `setRelationPairsCaseSensitive`

This parameter is set to determine whether relation pairs are case sensitive (Default: `False`).


In this example, the NER model contains `ADE` and `DRUG` entities (uppercased). We first set `relationPairs` to `["drug-ade, ade-drug"]` (lowercased) and `relationPairsCaseSensitive` to `True`.

In [19]:
ade_re_ner_chunk_filter = medical.RENerChunksFilter() \
    .setInputCols(["ner_chunks", "dependencies"])\
    .setOutputCol("re_ner_chunks")\
    .setMaxSyntacticDistance(10)\
    .setRelationPairs(["drug-ade, ade-drug"])\
    .setRelationPairsCaseSensitive(True)

pipeline = nlp.Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer,
    words_embedder,
    pos_tagger,
    ner_tagger,
    ner_chunker,
    dependency_parser,
    ade_re_ner_chunk_filter,
    ade_re_model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

In [20]:
example = """A 44-year-old man taking naproxen for chronic low back pain and a 20-year-old woman on oxaprozin for rheumatoid arthritis presented with tense bullae and cutaneous fragility on the face and the back of the hands."""

spark_df = spark.createDataFrame([[example]]).toDF("text")

results = model.transform(spark_df)

In [21]:
results.select(
    F.explode(F.arrays_zip(results.relations.result, results.relations.metadata)).alias("cols")
).select(
    F.expr("cols['0']").alias("relation"),
    F.expr("cols['1'].chunk1").alias("chunk1"),
    F.expr("cols['1'].entity1").alias("label1"),
    F.expr("cols['1'].chunk2").alias("chunk2"),
    F.expr("cols['1'].entity2").alias("label2"),
    F.expr("cols['1'].syntactic_distance ").alias("syntactic_distance ")
).show()

+--------+------+------+------+------+-------------------+
|relation|chunk1|label1|chunk2|label2|syntactic_distance |
+--------+------+------+------+------+-------------------+
+--------+------+------+------+------+-------------------+



As seen above, no relation was identified by the model because of the mismatch between the relation paris and the NER entities.

Now we will set `relationPairsCaseSensitive` to `False` and expect to see relations.

In [22]:
ade_re_ner_chunk_filter = medical.RENerChunksFilter() \
    .setInputCols(["ner_chunks", "dependencies"])\
    .setOutputCol("re_ner_chunks")\
    .setMaxSyntacticDistance(10)\
    .setRelationPairs(["drug-ade, ade-drug"])\
    .setRelationPairsCaseSensitive(False)

pipeline = nlp.Pipeline(stages=[
    documenter,
    sentencer,
    tokenizer,
    words_embedder,
    pos_tagger,
    ner_tagger,
    ner_chunker,
    dependency_parser,
    ade_re_ner_chunk_filter,
    ade_re_model
])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

In [23]:
results = model.transform(spark_df)

In [24]:
results.select(
    F.explode(F.arrays_zip(results.relations.result, results.relations.metadata)).alias("cols")
).select(
    F.expr("cols['0']").alias("relation"),
    F.expr("cols['1'].chunk1").alias("chunk1"),
    F.expr("cols['1'].entity1").alias("label1"),
    F.expr("cols['1'].chunk2").alias("chunk2"),
    F.expr("cols['1'].entity2").alias("label2"),
    F.expr("cols['1'].syntactic_distance ").alias("syntactic_distance ")
).show()

+--------+---------+------+--------------------+------+-------------------+
|relation|   chunk1|label1|              chunk2|label2|syntactic_distance |
+--------+---------+------+--------------------+------+-------------------+
|       1| naproxen|  DRUG|        tense bullae|   ADE|                  6|
|       1| naproxen|  DRUG|cutaneous fragili...|   ADE|                  7|
|       1|oxaprozin|  DRUG|        tense bullae|   ADE|                  3|
|       1|oxaprozin|  DRUG|cutaneous fragili...|   ADE|                  4|
+--------+---------+------+--------------------+------+-------------------+

